# 利用TOF中的$\gamma$峰，对TOF进行绝对刻度，并将飞行距离归一到100 cm，生成新的ntof

## 打开root文件，继承tree中Branch

In [1]:
%jsroot on
TCanvas *c1=new TCanvas("c1","c1");//创建画布

//声明branch变量
Double_t ctof,tof;
Int_t pid;
Double_t tu, td, x;

//打开root文件
TFile *ipf=new TFile("./tree.root");//打开ROOT文件
//得到tree的指针
TTree *tree=(TTree*)ipf->Get("tree");

//将tree中Branch与变量地址联系
tree->SetBranchAddress("ctof",&ctof);
tree->SetBranchAddress("tof",&tof);
tree->SetBranchAddress("pid",&pid);
tree->SetBranchAddress("tu",&tu);   
tree->SetBranchAddress("td",&td);
tree->SetBranchAddress("x",&x);

## 查看ctof

In [2]:
//%jsroot on
tree->Draw("ctof>>(1000,20,120)");
c1->Draw();

## 逐事件提取$\gamma$的ctof事件，ctof进行飞行距离的修正到100 cm,并加入offset因素

In [3]:
TH2D *corr_ctofvstx=new TH2D("corr_ctofvstx","corr_ctofvstx",240,-120,120,75,2.5,4);//ctof归一到100 cm且考虑offset影响后与tx关联
TH2D *corr_ctofvstx1=new TH2D("corr_ctofvstx1","corr_ctofvstx (all gamma)",240,-120,120,75,2.5,4);//ctof归一到100 cm且考虑offset影响(all gamma)后与tx关联

TH1D *corr_tof=new TH1D("corr_tof","corr_tof",200,0, 20);//模拟得到的tof归一到100 cm

TH1D *ntof=new TH1D("ntof","ntof",200,0, 20);//模拟测的得到的ctof归一到100 cm
TH1D *ntof1=new TH1D("ntof1","ntof all gamma",200,0, 20);//模拟测的得到的ctof归一到100 cm(all gamma)

TH1D *hce=new TH1D("hce","hce",500,0, 200);//模拟测的得到的ctof归一到100 cm
TH1D *hce1=new TH1D("hce1","hce all gamma",500,0, 200);//模拟测的得到的ctof归一到100 cm(all gamma)

//对事件进行遍历 
Long64_t nentries=tree->GetEntries();
for(Long64_t jentry=0; jentry<nentries; jentry++) 
    {
    tree->GetEntry(jentry);
    Double_t tx=3.742*(td-tu-14.915);  //时间差法刻度的位置
    Double_t d=TMath::Sqrt(502.5*502.5+tx*tx);//考虑了探测器的厚度的飞行距离
    Double_t corr_ctof=(ctof-26.2096)/d*100.;//不同距离的tof统一修正到100 cm
    corr_ctofvstx->Fill(tx, corr_ctof);
    
    Double_t C=d/30.-(42.9542+1.95083e-6*tx+3.20798e-5*tx*tx);
    Double_t corr_ctof1=(ctof+C)/d*100.;//不同距离的tof统一修正到100 cm all gamma
    corr_ctofvstx1->Fill(tx, corr_ctof1);
    
    ntof->Fill(corr_ctof);
    ntof1->Fill(corr_ctof1);
    corr_tof->Fill(tof*100./d);
    if( pid ==1)
        {
        Double_t ce=(939.56563*100.*100.)/(2.*30.*30.*corr_ctof*corr_ctof);
        hce->Fill(ce);
        Double_t ce1=(939.56563*100.*100.)/(2.*30.*30.*corr_ctof1*corr_ctof1);
        hce1->Fill(ce1);
        }
  /*  else
    {
        Double_t ce=0.;
        hce->Fill(ce);
        Double_t ce1=0.;
        hce1->Fill(ce1);
     }*/
    }
corr_ctofvstx->Draw("colz");
c1->Draw();

In [4]:
corr_ctofvstx1->Draw("colz");
c1->Draw();

## 对比模拟实验测的corr_ctof与模拟得到的corr_tof

In [5]:
ntof->Draw();
corr_tof->SetLineColor(kRed);
corr_tof->Draw("same");
c1->SetLogy();
c1->Draw();

In [6]:
ntof1->Draw();
corr_tof->SetLineColor(kRed);
corr_tof->Draw("same");
c1->SetLogy();
c1->Draw();

## 中子能量

In [7]:
hce->Draw();
hce->Fit("gaus");
c1->SetLogy(0);
c1->Draw();

 FCN=369.128 FROM MIGRAD    STATUS=CONVERGED      62 CALLS          63 TOTAL
                     EDM=4.56152e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     5.24049e+02   2.43977e+00   1.86536e-02   2.42113e-05
   2  Mean         1.00482e+02   8.06164e-02   7.57141e-04   3.69568e-03
   3  Sigma        2.12171e+01   5.76762e-02   6.90309e-06   4.66810e-02


- 模拟输入量：$E_n = 100 MeV$,$FWHM = 50 MeV$;
- 数据处理结果：$E_{mean}=100.482 MeV$,$sigma = 21.2171 , FWHM = 2.355* sigma= 49.96 MeV$，与模拟结果基本对应。

In [8]:
hce1->Draw();
hce1->Fit("gaus");
c1->SetLogy(0);
c1->Draw();

 FCN=367.766 FROM MIGRAD    STATUS=CONVERGED      52 CALLS          53 TOTAL
                     EDM=4.07178e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     5.24120e+02   2.44431e+00   1.86216e-02   5.49565e-05
   2  Mean         1.00431e+02   8.06292e-02   7.55870e-04   3.25372e-03
   3  Sigma        2.12158e+01   5.79957e-02   6.91932e-06   1.10256e-01


In [9]:
ipf->Close();//关闭root文件

In [10]:
!jupyter nbconvert homework1.1-4 --to html

[NbConvertApp] Converting notebook homework1.1-4.ipynb to html

[NbConvertApp] Writing 451533 bytes to homework1.1-4.html

